 ### Form Submition:

 1. Navigate to you post-detail.html template
 2. Set the method of the comment form to POST
 3. The action?
    - We want to send the form to the same page but this time with a post request!
    - So use the url tag and provide it with the name of the url.
    - you should also send post.slug to the url

Your form should look like the following block:

In [ ]:
<form action='{% url "blog-post-detail" post.slug %}' method="post">

At the moment we dont have a way of handeling the PostDetailView.

4. To fix this we need to redefine our PostDetailView
    - instead of extending it from DetailView, extend it from View
    - Define post and get method
    - Delete the get_context... method
    - Manulay handle post and get

Your PostDetailView should look like the following block:

In [ ]:
from django.shortcuts import render
from .fomrs import CommentForm
from .models import Post
from django.http import HttpResponseRedirect
from django.urls import reverse

class PostDetailView(View):
    model = Post
    template_name = "blog/post-detail.html"

    def get(self, request, slug):
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':CommentForm()
        })

    def post(self, request, slug):
        comment_form = CommentForm(request.POST)
        if comment_form.is_valid():
            comment_form.save()
            return HttpResponseRedirect(reverse('blog-post-detail', args=[slug]))
        
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':comment_form
        })

We have a problem here:

Since we do not use the whole fomr to construct our form in the template (the post field is excluded), we can not save it the way it is and we need to modify it before saving it<br>

We can fix this by adding an extra input argument to the .save method.<br>
This extra input argument is called commit and we pass False.<br>
This tells django that instead of saving the form to the database, it has to create an object from it (pass it to a new variable).<br>
Now, we can edit it before saving it to the database.<br>
To do this you just need to pass the post to it like it is an atrribute of the new object form.<br>

Your PostDetailView should look like the following block:

In [ ]:
class PostDetailView(View):
    model = Post
    template_name = "blog/post-detail.html"

    def get(self, request, slug):
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':CommentForm()
        })

    def post(self, request, slug):
        comment_form = CommentForm(request.POST)
        post = Post.objects.get(slug = slug)

        if comment_form.is_valid():
            comment = comment_form.save(commit=False)
            comment.post = post
            comment.save()
            return HttpResponseRedirect(reverse('blog-post-detail', args=[slug]))
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':comment_form
        })

Now, everything should work just fine...<br>
The only thing that we need to do is to display the comments under each post.<br>
We will do it in the next notebook<br>


Before we move to the next notebook modify the admin.py file so that we can see all the comments in the administrative section.<br>

Also, style the error messages.

Your post-detail.html template should look like the following block:

In [ ]:
{% extends "base.html" %}
{% load static %}

{% block title %}{{ post.title }}{% endblock %}

{% block css_files %}
    <link rel="stylesheet" href="{% static "blog/post-detail.css" %}">
{% endblock %}

{% block content %}
    {% if form.errors %}
        <div class='error-box'>
            <h3>Error</h3>
            <h4>Sorry, we couldn't submit your comment to the server.</h4>
            <p>Please check your inputs and resubmit the form.</p>
        </div>
    {% endif %}

    <section id='summary'>
        <h2>{{ post.title }}</h2>
        <p>
            {% for tag in post_tags %}
                <span class='tag'>{{ tag.caption }} </span>
            {% endfor %}
        </p>
        <article>
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
            <address>By <a href='mailto:{{ post.author.email }}'>{{ post.author }}</a></address>
            <div>
                Last Update on <time>{{ post.date|date:"d M Y" }}</time>
            </div>
        </article>
    </section>
    <main>
        <p>
            {{ post.content }}
        </p>
    </main>
    <div class='comments-section'>
        <form action='{% url "blog-post-detail" post.slug %}' method="post">
            {% csrf_token %}
            <div class='top-div'>
                <div class='top-left'>
                    {{ form.user_name.label_tag }}
                    {% if form.user_name.errors %}
                        {{ form.user_name.errors}}
                    {% endif %}
                    {{ form.user_name }}
                    
                    {{ form.user_email.label_tag }}
                    {% if form.user_email.errors %}
                        {{ form.user_emaile.errors}}
                    {% endif %}
                    {{ form.user_email }}
                </div>
                <div class='top-right'>
                    <button type="submit">Submit Comment</button>
                </div>
            </div>
            <div class="bottom-div">
                {{ form.text.label_tag }}
                {% if form.text.errors %}
                        {{ form.text.errors}}
                    {% endif %}
                {{ form.text }}
            </div>
        </form>
    </div>
{% endblock %}

Your post-detail.css file should look like the fowllowing block:

In [ ]:
body{
    background-color:#e7e7e7;
}

#main-navigation{
    background: linear-gradient(to right top, #338dc4, #0bb5e9);
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.25);
}

#summary{
    margin: 8rem auto 3rem auto;
    padding:2rem;
    width:90%;
    max-width: 90rem;
    border-radius: 12px;
    background-color: #15753f;
    position: relative;
    box-shadow: 0 2px 8px rgba(0,0,0,0.45);
    height:22rem;
}

#summary h2{
    color:white;
    font-size:4.5rem;
    margin-bottom:-0.25rem;
}

#summary article{
    position:absolute;
    top:2rem;
    right: 3rem;
    padding: 1rem 2rem;
    border-radius: 12px;
    text-align: right;
    color: white;
}

#summary article img{
    width:12rem;
    height:12rem;
    border-radius: 12px;
    transform: rotate(5deg);
    border:5px solid white;
    margin-bottom: 1rem;
}

#summary:hover,
#summary:active{
    background-color:#338dc4;
}

#summary address{
    font-weight: bold;
}

#summary time{
    font-weight: bold;
}

#summary a{
    color:white;
}
#summary .tag{
    color:inherit;
    background-color: white;
    padding:0.3rem 1rem;
    border-radius: 8px;
    margin-left:5px;
}
main{
    margin:3rem auto 0 auto;
    background-color:white;
    width:90%;
    max-width: 90rem;
    padding:2rem;
    border-radius: 16px;
    box-shadow: 0 2px 12px rgba(0,0,0,0.45);
}
.comments-section{
    width:90%;
    max-width: 90rem;
    padding: 2rem;
    border-radius: 16px;
    background-color:rgba(101, 13, 138, 0.8);
    margin:4rem auto 0 auto;
}
.comments-section form{
    width:50%;
    max-width: 50rem;
    margin:auto;
    padding:2rem;
    border:1px solid rgba(255,255,255,0.2);
    border-radius: 16px;
    box-shadow: 0px 0px 5px rgb(255,255,255);
    margin-top:2rem;
}
.top-div {
    display: flex;
    justify-content: space-between;
    align-items: center; /* Align items vertically centered */
}

.top-left {
    display: flex;
    flex-direction: column;
    margin-bottom: 0.5rem;
}
.top-left label{
    display:block;
    color:white;
}
.top-left input{
    width:30rem;
    margin-bottom:1rem;
    height:2rem;
    border-radius: 6px;
    border:2px solid gray;
}
.top-left input:hover,
.top-left input:focus{
    border-color:#338dc4;
}
.top-right button{
    width:8rem;
    height:8rem;
    border-radius: 8px;
    border:1px solid transparent;
    background-color:#338dc4;
    color:white;
}
.top-right button:hover,
.top-right button:focus{
    cursor: pointer;
    background-color:#0bb5e9;
}
.bottom-div{
    display: block;
    width:100%;
}
.bottom-div label{
    display:block;
    color:white;
}
.bottom-div textarea{
    width:100%;
    resize: none;
    height: 10rem;
    padding:1rem;
    font-size:1.2rem;
}
form .errorlist{
    color:white;
}
form .errorlist li{
    padding:0.5rem;
    background-color:rgb(162, 27, 27);
    border-radius: 8px;
    display: inline;
}
.error-box{
    width:40rem;
    background-color: rgb(198, 57, 57);
    border:1px solid rgb(209, 45, 45);
    color:white;
    padding:1.5rem;
    margin:10rem auto 2rem auto;
}